In [ ]:
#import os
import numpy as np
import torch

from lib.config.project_manager import ProjectManager
from lib.dataset.dataset2D import Dataset2D
from lib.hybridnet.efficienttrack.efficienttrack import EfficientTrack

project = ProjectManager()
project.load('Test_Project')

#project.create_new(
#    name = 'Test_Project2',
#    dataset2D_path = '/home/timo/Desktop/HybridNet/datasets/Ralph_All',
#    dataset3D_path = '/home/timo/Desktop/HybridNet/datasets/Ralph_All')

cfg = project.get_cfg()

In [ ]:
training_set = Dataset2D(project.cfg, set='train', mode = 'CenterDetect')
val_set = Dataset2D(project.cfg, set='val',mode = 'CenterDetect')

In [ ]:
weights = '/home/timo/Desktop/HybridNet/projects/Test_Project/models/CenterDetect/Run_20211120-151850/EfficientTrack-d0_10.pth'

centerDetect = EfficientTrack('CenterDetect', cfg, None)

In [ ]:
centerDetect.train(training_set, val_set, 1000)

In [ ]:
import lib.hybridnet.modules.efficienttrack.darkpose as darkpose
import cv2
import matplotlib.pyplot as plt

weights = "/home/timo/Desktop/HybridNet/projects/Test_Project/models/CenterDetect/Run_20211120-151850/EfficientTrack-d0_10.pth"

centerDetect = EfficientTrack('CenterDetectInference', cfg, weights)

In [ ]:
j = 0
item = val_set.__getitem__(j)
j += 20
imgs = item[0]
print (imgs.shape)
img = ((imgs*cfg.DATASET.STD)+cfg.DATASET.MEAN)*255

imgs = torch.from_numpy(imgs).permute(2, 0, 1).float()

imgs = imgs.reshape(1,3,256,320)
imgs = imgs.cuda()
outputs = centerDetect.model(imgs)
preds, maxvals = darkpose.get_final_preds(outputs[1].clamp(0,255).detach().cpu().numpy(), None)

print (maxvals)

colors = [(255,0,0), (255,0,0),(255,0,0),(255,0,0),(0,255,0),(0,255,0),(0,255,0),(0,255,0),(0,0,255),(0,0,255),(0,0,255),(0,0,255),(255,255,0),(255,255,0),(255,255,0), (255,255,0),
                  (0,255,255),(0,255,255),(0,255,255),(0,255,255), (255,0,255),(100,0,100),(100,0,100)]
for i,point in enumerate(preds[0]):
    if (maxvals[0][i]) > 10:
        cv2.circle(img, (int(point[0]*2), int(point[1])*2), 2, (255,0,0), thickness=5)
    else:
        cv2.circle(img, (int(point[0]*2), int(point[1])*2), 2, (100,0,0), thickness=5)

plt.imshow(img/255.)
hetamap = cv2.resize(outputs[1].clamp(0,255).detach().cpu().numpy()[0][0]/255., (320,256), interpolation=cv2.cv2.INTER_NEAREST)
plt.imshow(hetamap, alpha=0.4)
plt.imsave( 'test.png',img/255.)
#plt.savefig('test.png')